In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_csv('Data/Real-Data/Real_Combine.csv')
df.head()

In [ ]:
sns.heatmap(df.isnull(), yticklabels=False, cbar = False, cmap = 'viridis')

In [ ]:
df.isnull().sum()

In [ ]:
df= df.dropna()

In [ ]:
# Split labels and data

X=df.iloc[:,:-1] ## independent features
y=df.iloc[:,-1] ## dependent features

In [ ]:
print('Null Value in X is \n{}'.format(X.isnull().sum()))
print('Null Value in y is \n{}'.format(y.isnull().sum()))

In [ ]:
sns.pairplot(df)

In [ ]:
# Checking Correlation among the variable 
corr_mat = df.corr()

In [ ]:
plt.figure(figsize=(20,20))
sns.heatmap(corr_mat, cmap = 'coolwarm',annot= True)


### Feature Importance 
Using Extra Tree Regressor

In [ ]:
from sklearn.ensemble import ExtraTreesRegressor
model = ExtraTreesRegressor()
model.fit(X,y)

In [ ]:
#plot graph of feature importances for better visualization

feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(5).plot(kind='barh')
plt.show()

In [ ]:
X.head()

In [ ]:
# We can see that the distribution for this is right skewed. Most of the values lie
# on the right part of the distribution
sns.distplot(y)

#### Train Test Split

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.3, random_state = 0)

In [ ]:
X_train.shape

In [ ]:
X_test.shape

### Random Forest Regressor

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
regressor = RandomForestRegressor()
regressor.fit(X_train, y_train)

In [ ]:
print('R^2 score in Training Set {}'.format(regressor.score(X_train, y_train)))

In [ ]:
print('R^2 score is Test Set {}'.format(regressor.score(X_test, y_test)))

In [ ]:
from sklearn.model_selection import cross_val_score
score = cross_val_score(regressor, X,y, cv =5)

In [ ]:
score.mean()

#### Model Evaluation

In [ ]:
prediction = regressor.predict(X_test)

In [ ]:
sns.distplot(y_test - prediction)

In [ ]:
plt.scatter(y_test , prediction)

#### Hyperparameter Tuning Decision Tree Regressor

In [ ]:
#Randomized Search CV

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 1200, num = 12)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(5, 30, num = 6)]
# max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10, 15, 100]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 5, 10]
# Method of selecting samples for training each tree
# bootstrap = [True, False]

In [ ]:
# Create the random grid
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf}

print(random_grid)

In [ ]:
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
rf = RandomForestRegressor()

In [ ]:
random_search= RandomizedSearchCV(estimator = rf,param_distributions= random_grid,scoring='neg_mean_squared_error',n_jobs=-1,cv=5,verbose=2
                                  , n_iter = 100,random_state= 45)

In [ ]:
random_search.fit(X_train, y_train)
%time

In [ ]:
random_search.best_params_

In [ ]:
predictions=random_search.predict(X_test)

In [ ]:
sns.distplot(y_test - predictions)


#### Regression Evaluation Metrics
Here are three common evaluation metrics for regression problems:

Mean Absolute Error (MAE) is the mean of the absolute value of the errors:

$$\frac 1n\sum_{i=1}^n|y_i-\hat{y}_i|$$
Mean Squared Error (MSE) is the mean of the squared errors:

$$\frac 1n\sum_{i=1}^n(y_i-\hat{y}_i)^2$$
Root Mean Squared Error (RMSE) is the square root of the mean of the squared errors:

$$\sqrt{\frac 1n\sum_{i=1}^n(y_i-\hat{y}_i)^2}$$
Comparing these metrics:

MAE is the easiest to understand, because it's the average error.
MSE is more popular than MAE, because MSE "punishes" larger errors, which tends to be useful in the real world.
RMSE is even more popular than MSE, because RMSE is interpretable in the "y" units.
All of these are loss functions, because we want to minimize them.

In [ ]:
from sklearn import metrics

In [ ]:
print('MAE:', metrics.mean_absolute_error(y_test, predictions))
print('MSE:', metrics.mean_squared_error(y_test, predictions))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, predictions)))

In [ ]:
import pickle
import os

In [ ]:
if not os.path.exists("Models"):
        os.makedirs("Models")
os.chdir("Models")

In [ ]:
# open a file, where you ant to store the data
file = open('decision_regression_model.pkl', 'wb')

# dump information to that file
pickle.dump(random_search, file)